In [1]:
import os
import sys
import importlib

# Thêm đường dẫn cho các module
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

# Import các module
import import_default
import import_database  
import import_other
import clean_data
import candle_chart

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(clean_data)
importlib.reload(candle_chart)

from import_default import *
from import_database import *
from import_other import *
from clean_data import *
from candle_chart import *

In [2]:
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')
current_quarter_classification_df = get_mongo_collection(ref_db, 'current_quarter_classification')

In [69]:
date_series = get_mongo_collection(ref_db, 'date_series')
today = date_series.iloc[0]['date']
projection={"_id": 0, 'date': 1, 'ticker': 1, 'vol_ratio': 1, 'W_MTSI': 1, 'M_MTSI': 1, 'Q_MTSI': 1, 'Y_MTSI': 1, 'W_MRVI': 1, 'M_MRVI': 1, 'Q_MRVI': 1, 'Y_MRVI': 1}

today_stock_df = get_mongo_collection(stock_db, 'today_stock', projection=projection)
history_stock_df = get_mongo_collection(stock_db, 'history_stock', projection=projection,
                                        find_query={"date": {"$in": date_series['date'].iloc[:5].tolist()}})
full_stock_df = pd.concat([today_stock_df, history_stock_df])
stock_list = ["SSI", "VIX", "VND", "VDS", "HCM", "SHS", "MBS", "CTS", "ORS", "VCI", "FTS", "BSI", "BVS"]

In [70]:
vol_ratio_df = full_stock_df[['date', 'ticker', 'vol_ratio']].copy()
vol_ratio_df['date'] = vol_ratio_df['date'].astype(str)

vol_ratio_df = vol_ratio_df.pivot(index="ticker", columns="date", values="vol_ratio")
vol_ratio_df['total_ratio'] = vol_ratio_df.sum(axis=1)
vol_ratio_df = vol_ratio_df[vol_ratio_df.index.isin(stock_list)].sort_values(by='total_ratio', ascending=False).reset_index()

In [66]:
mrvi_df = full_stock_df[['date', 'ticker', 'W_MRVI', 'M_MRVI', 'Q_MRVI', 'Y_MRVI']].copy()
mrvi_df = mrvi_df[mrvi_df['date'] == today].drop(columns=['date'])
mrvi_df = mrvi_df[mrvi_df['ticker'].isin(stock_list)]

In [71]:
mtsi_df = full_stock_df[['date', 'ticker', 'W_MTSI', 'M_MTSI', 'Q_MTSI', 'Y_MTSI']].copy()
mtsi_df = mtsi_df[mtsi_df['date'] == today].drop(columns=['date'])
mtsi_df = mtsi_df[mtsi_df['ticker'].isin(stock_list)]

In [73]:
with pd.ExcelWriter('../output/other/custom.xlsx') as writer:
    vol_ratio_df.to_excel(writer, sheet_name='Vol_Ratio', index=False)
    mrvi_df.to_excel(writer, sheet_name='MRVI', index=False)
    mtsi_df.to_excel(writer, sheet_name='MTSI', index=False)